In [1]:
import composable_mapping as cm
import torch

In [21]:
sampler = cm.LinearInterpolator(
    mask_extrapolated_regions_for_empty_volume_mask=False, extrapolation_mode="border"
)
volume = torch.tensor(
    [
        [1.0, -1.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 2.0, 0.0],
    ]
)[None, None]
coordinate_system = cm.CoordinateSystem.voxel((3, 3))
mapping = cm.samplable_volume(
    volume,
    coordinate_system=coordinate_system,
    sampler=sampler,
)
mapping_to_sample = mapping.resample(
    sampler=sampler.derivative(spatial_dim=0, limit_direction=cm.LimitDirection.right()),
)

In [22]:
mapping_to_sample.sample_to((cm.CoordinateSystem.voxel((1, 1))) + [1.0, 1.0]).generate_values()

tensor([[[[1.]]]])